In [1]:
!pip install evaluate
!pip install seqeval
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=b4faa8073ac5d70b88c2dd1f5e41a53717153b09b1e8b455dce2b5a5668a8e08
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [14]:
from transformers import BertTokenizerFast
import tensorflow as tf

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
import json
with open("/kaggle/input/cve-1-0/labeled_dataset.json", "r") as f: dataset = json.load(f)

In [17]:
import re

def trim_non_alphanumeric(s):
    return re.sub(r'^\W+|\W+$', '', s)

In [18]:
vocab = set()
for V in dataset:
    tokens = V["description"].split(" ")
    tokens = [trim_non_alphanumeric(token) for token in tokens]
    vocab.update(tokens)
vocab = list(vocab)[1:]

In [19]:
vocab.sort()

In [20]:
states = ["B-ENV", "I-ENV", "B-ATTACK", "I-ATTACK", "B-ATTACK_VECTOR", "I-ATTACK_VECTOR", "B-ATTACK_TYPE", "I-ATTACK_TYPE", "B-ATTACK_DESCRIPTION", "I-ATTACK_DESCRIPTION", "B-PRE_REQ", "I-PRE_REQ", "B-OUTCOME", "I-OUTCOME", "O"]
n_states = len(states)
n_states

15

In [21]:
id2state = {}
state2id = {}

for i, state in enumerate(states):
    id2state[i] = state
    state2id[state] = i

In [22]:
id2state, state2id

({0: 'B-ENV',
  1: 'I-ENV',
  2: 'B-ATTACK',
  3: 'I-ATTACK',
  4: 'B-ATTACK_VECTOR',
  5: 'I-ATTACK_VECTOR',
  6: 'B-ATTACK_TYPE',
  7: 'I-ATTACK_TYPE',
  8: 'B-ATTACK_DESCRIPTION',
  9: 'I-ATTACK_DESCRIPTION',
  10: 'B-PRE_REQ',
  11: 'I-PRE_REQ',
  12: 'B-OUTCOME',
  13: 'I-OUTCOME',
  14: 'O'},
 {'B-ENV': 0,
  'I-ENV': 1,
  'B-ATTACK': 2,
  'I-ATTACK': 3,
  'B-ATTACK_VECTOR': 4,
  'I-ATTACK_VECTOR': 5,
  'B-ATTACK_TYPE': 6,
  'I-ATTACK_TYPE': 7,
  'B-ATTACK_DESCRIPTION': 8,
  'I-ATTACK_DESCRIPTION': 9,
  'B-PRE_REQ': 10,
  'I-PRE_REQ': 11,
  'B-OUTCOME': 12,
  'I-OUTCOME': 13,
  'O': 14})

In [23]:
X = []
y = []
for V in dataset:
    try:
        annotated_tokens = V["labeled_description"]
        tokens = []
        labels = []
        for token in annotated_tokens:    
            if (token[0] == ''):
                continue
            if (len(token) != 2):
                continue
            t = trim_non_alphanumeric(token[0])
            if (t == ''):
                continue
            tokens.append(t)
            labels.append(token[1])
        X.append(tokens)
        y.append(labels)
    except Exception as e:
            pass

In [24]:
example = X[0]
example

['A',
 'cross-site',
 'scripting',
 'XSS',
 'vulnerability',
 'in',
 'ONLYOFFICE',
 'Document',
 'Server',
 'Example',
 'before',
 'v7.0.0',
 'allows',
 'remote',
 'attackers',
 'inject',
 'arbitrary',
 'HTML',
 'or',
 'JavaScript',
 'through',
 'example/editor']

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [30]:
tokenizer = BertTokenizerFast.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
tokenized_input = tokenizer(example, is_split_into_words=True)
tokenized_input

{'input_ids': [101, 1037, 2892, 1011, 2609, 5896, 2075, 1060, 4757, 18130, 1999, 2069, 7245, 6610, 6254, 8241, 2742, 2077, 1058, 2581, 1012, 1014, 1012, 1014, 4473, 6556, 17857, 1999, 20614, 15275, 16129, 2030, 9262, 22483, 2083, 2742, 1013, 3559, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'a',
 'cross',
 '-',
 'site',
 'script',
 '##ing',
 'x',
 '##ss',
 'vulnerability',
 'in',
 'only',
 '##off',
 '##ice',
 'document',
 'server',
 'example',
 'before',
 'v',
 '##7',
 '.',
 '0',
 '.',
 '0',
 'allows',
 'remote',
 'attackers',
 'in',
 '##ject',
 'arbitrary',
 'html',
 'or',
 'java',
 '##script',
 'through',
 'example',
 '/',
 'editor',
 '[SEP]']

In [17]:
word_ids = tokenized_input.word_ids()
word_ids

[None,
 0,
 1,
 1,
 1,
 2,
 2,
 3,
 3,
 4,
 5,
 6,
 6,
 6,
 7,
 8,
 9,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 13,
 14,
 15,
 15,
 16,
 17,
 18,
 19,
 19,
 20,
 21,
 21,
 21,
 None]

In [31]:
def tokenize_and_align_labels(X, y):
    tokenized_inputs = tokenizer(X, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(y):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(state2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [32]:
dataset_train = tokenize_and_align_labels(X_train, y_train)
dataset_val = tokenize_and_align_labels(X_val, y_val)

In [46]:
dataset_test = tokenize_and_align_labels(X_test, y_test)

In [33]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [34]:
import evaluate

In [35]:
seqeval = evaluate.load("seqeval")

In [36]:
import numpy as np

In [37]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [states[p] for (p, l) in zip(prediction, label) if l != -100]

        for prediction, label in zip(predictions, labels)

    ]

    true_labels = [

        [states[l] for (p, l) in zip(prediction, label) if l != -100]

        for prediction, label in zip(predictions, labels)

    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [38]:
from transformers import TFAutoModelForTokenClassification, create_optimizer

In [ ]:
model = TFAutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=n_states, id2label=id2state, label2id=state2id
)

In [39]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [62]:
batch_size = 16
num_train_epochs = 20
num_train_steps = (len(X_train) // batch_size) * num_train_epochs


In [63]:
num_train_steps

1800

In [64]:
from datasets import Dataset

In [65]:
tf_train_set = model.prepare_tf_dataset(
    Dataset.from_dict(dataset_train),
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    Dataset.from_dict(dataset_val),
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [108]:
tf_test_set = model.prepare_tf_dataset(
    Dataset.from_dict(dataset_test),
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [67]:
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

Number of devices: 1


In [68]:
import os
from tensorflow import keras

# Prepare a directory to store all the checkpoints.
checkpoint_dir = "./ckpt"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)


In [69]:
def get_compiled_model():
    model = TFAutoModelForTokenClassification.from_pretrained(
        "distilbert/distilbert-base-uncased", num_labels=n_states, id2label=id2state, label2id=state2id
    )
    optimizer, lr_schedule = create_optimizer(
        init_lr=2e-5,
        num_train_steps=num_train_steps,
        weight_decay_rate=0.01,
        num_warmup_steps=0,
    )
    model.compile(optimizer=optimizer)
    return model

In [70]:
def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model()


In [71]:
with strategy.scope():
    model = make_or_restore_model()

Creating a new model


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able t

In [72]:
model.summary()

Model: "tf_distil_bert_for_token_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 dropout_59 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  11535     
                                                                 
Total params: 66374415 (253.20 MB)
Trainable params: 66374415 (253.20 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
from transformers.keras_callbacks import KerasMetricCallback

In [74]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [75]:
callbacks = [metric_callback, keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_dir + "/ckpt-{epoch}", save_freq="epoch"
        )]

In [70]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_train_epochs, callbacks=callbacks)

Epoch 1/20
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
113/113 [==============================] - ETA: 0s - loss: 1.1944

/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


113/113 [==============================] - 171s 892ms/step - loss: 1.1944 - val_loss: 0.6750 - precision: 0.4256 - recall: 0.4777 - f1: 0.4501 - accuracy: 0.8049
Epoch 2/20
113/113 [==============================] - ETA: 0s - loss: 0.5735

/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


113/113 [==============================] - 65s 578ms/step - loss: 0.5735 - val_loss: 0.5116 - precision: 0.4871 - recall: 0.5653 - f1: 0.5233 - accuracy: 0.8352
Epoch 3/20
113/113 [==============================] - ETA: 0s - loss: 0.4440

...

113/113 [==============================] - 68s 606ms/step - loss: 0.1514 - val_loss: 0.5524 - precision: 0.5295 - recall: 0.6230 - f1: 0.5725 - accuracy: 0.8496
Epoch 19/20
113/113 [==============================] - ETA: 0s - loss: 0.1476

/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


113/113 [==============================] - 70s 626ms/step - loss: 0.1476 - val_loss: 0.5567 - precision: 0.5214 - recall: 0.6282 - f1: 0.5699 - accuracy: 0.8495
Epoch 20/20
113/113 [==============================] - ETA: 0s - loss: 0.1463

/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


113/113 [==============================] - 68s 608ms/step - loss: 0.1463 - val_loss: 0.5564 - precision: 0.5219 - recall: 0.6265 - f1: 0.5694 - accuracy: 0.8494


In [76]:
# model = keras.models.load_model("/kaggle/working/latest_ckpt/")
model.load_weights("/kaggle/working/latest_ckpt")

In [77]:
model.evaluate(tf_test_set)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
29/29 [==============================] - 118s 3s/step - loss: 0.5513


0.5512956380844116

In [100]:
dataset_test = tokenize_and_align_labels(X_test, y_test)
dataset_test.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [96]:
len(dataset_test["input_ids"][0]), len(dataset_test["labels"][0]), len(dataset_test["attention_mask"][0])

(63, 63, 63)

In [97]:
len(X_test)

454

In [118]:
len(dataset_test["labels"])

454

In [128]:
def predict(text):
    text_tokenized = tokenizer(text,is_split_into_words=True, truncation=True, return_tensors="tf")
    del text_tokenized['token_type_ids']
    logits = model(**text_tokenized).logits
    predicted_token_class_ids = tf.math.argmax(logits, axis=-1)
    predicted_token_class = [(tokenizer.decode(token), model.config.id2label[t]) for token, t in zip( text_tokenized["input_ids"][0], predicted_token_class_ids[0].numpy().tolist() )]
    # print(predicted_token_class)
    return predicted_token_class

(63, 63)

In [142]:
pred = []
true_states = []
for i in range(len(X_test)):
    
    pred.append(predict(X_test[i]))
    true_states.append(dataset_test['labels'][i])
    

In [170]:
true_predictions = [
        [p[1] for (p, l) in zip(prediction, label) if l != -100]

        for prediction, label in zip(pred, true_states)

    ]

In [171]:
true_labels = [

        [id2state[l] for (p, l) in zip(prediction, label) if l != -100]

        for prediction, label in zip(pred, true_states)

    ]


In [175]:
results = seqeval.compute(predictions=true_predictions, references=true_labels)
print({
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]})

{'precision': 0.5218648355619805, 'recall': 0.6264642082429501, 'f1': 0.5694006309148264, 'accuracy': 0.8494004542813375}


In [71]:
model.save("BERT_SEQ_CLASSIFICATION.keras")

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [73]:
sample_text = "Opencats v0.9.7 was discovered to contain a SQL injection vulnerability via the importID parameter in the Import viewerrors function."

In [84]:
sample_text_tokenized = tokenizer(sample_text, truncation=True, return_tensors="tf")
del sample_text_tokenized['token_type_ids']

In [93]:
sample_text_tokenized["input_ids"][0]

<tf.Tensor: shape=(31,), dtype=int32, numpy=
array([  101,  2330, 19588,  1058,  2692,  1012,  1023,  1012,  1021,
        2001,  3603,  2000,  5383,  1037, 29296, 13341, 18130,  3081,
        1996, 12324,  3593, 16381,  1999,  1996, 12324, 13972, 29165,
        2015,  3853,  1012,   102], dtype=int32)>

In [85]:
logits = model(**sample_text_tokenized).logits

In [86]:
logits.shape

TensorShape([1, 31, 15])

In [94]:
predicted_token_class_ids = tf.math.argmax(logits, axis=-1)
predicted_token_class = [(tokenizer.decode(token), model.config.id2label[t]) for token, t in zip( sample_text_tokenized["input_ids"][0], predicted_token_class_ids[0].numpy().tolist() )]

In [95]:
predicted_token_class

[('[CLS]', 'O'),
 ('open', 'B-ENV'),
 ('##cats', 'I-ENV'),
 ('v', 'I-ENV'),
 ('##0', 'I-ENV'),
 ('.', 'I-ENV'),
 ('9', 'I-ENV'),
 ('.', 'I-ENV'),
 ('7', 'I-ENV'),
 ('was', 'O'),
 ('discovered', 'O'),
 ('to', 'O'),
 ('contain', 'O'),
 ('a', 'O'),
 ('sql', 'B-ATTACK'),
 ('injection', 'I-ATTACK'),
 ('vulnerability', 'O'),
 ('via', 'O'),
 ('the', 'O'),
 ('import', 'B-ATTACK_VECTOR'),
 ('##id', 'I-ATTACK_VECTOR'),
 ('parameter', 'I-ATTACK_VECTOR'),
 ('in', 'O'),
 ('the', 'O'),
 ('import', 'B-ATTACK_VECTOR'),
 ('viewer', 'I-ATTACK_VECTOR'),
 ('##ror', 'I-ATTACK_VECTOR'),
 ('##s', 'I-ATTACK_VECTOR'),
 ('function', 'I-ATTACK_VECTOR'),
 ('.', 'O'),
 ('[SEP]', 'O')]

In [96]:
!rm -rf ./ckpt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [97]:
import pickle

In [101]:
with open('/kaggle/working/tokenizer.pkl', "wb") as f: pickle.dump(tokenizer, f, pickle.HIGHEST_PROTOCOL)

In [104]:
!zip -r BERT_FOR_SEQUENCE_CLASSIFICATION.zip ./*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: BERT_SEQ_CLASSIFICATION.keras (deflated 2%)
  adding: latest_ckpt/ (stored 0%)
  adding: latest_ckpt/keras_metadata.pb (deflated 94%)
  adding: latest_ckpt/variables/ (stored 0%)
  adding: latest_ckpt/variables/variables.index (deflated 80%)
  adding: latest_ckpt/variables/variables.data-00000-of-00001 (deflated 26%)
  adding: latest_ckpt/saved_model.pb (deflated 93%)
  adding: latest_ckpt/fingerprint.pb (stored 0%)
  adding: latest_ckpt/assets/ (stored 0%)
  adding: tokenizer.pkl (deflated 58%)


In [105]:
!rm ./BERT_FOR_SEQUENCE_CLASSIFICATION.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [41]:
from tensorflow import keras

In [42]:
model = TFAutoModelForTokenClassification.from_pretrained(
        "distilbert/distilbert-base-uncased", num_labels=n_states, id2label=id2state, label2id=state2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able t

In [43]:
# model = keras.models.load_model("/kaggle/working/latest_ckpt/")
model.load_weights("/kaggle/working/latest_ckpt")

In [44]:
def predict(text):
    text_tokenized = tokenizer(text, truncation=True, return_tensors="tf")
    del text_tokenized['token_type_ids']
    logits = model(**text_tokenized).logits
    predicted_token_class_ids = tf.math.argmax(logits, axis=-1)
    predicted_token_class = [(tokenizer.decode(token), model.config.id2label[t]) for token, t in zip( text_tokenized["input_ids"][0], predicted_token_class_ids[0].numpy().tolist() )]
    return predicted_token_class

In [46]:
for V in dataset:
    V["predicted_description"]= predict(V["description"])

In [47]:
import json

In [50]:
with open("/kaggle/working/predicted_bert.json", "w") as f: f.write(json.dumps(dataset, indent=4))